In [1]:
import json
from multiprocessing import Process

# Step 1: get protein data from fasta file
def gen_seqs(filename):
    seq = ''
    with open(filename) as f:
        for line_no, l in enumerate(f):
            if line_no == 0:
                pass
            elif l[0] == '>':
                yield seq
                seq = ''
            else:
                seq = seq + l.strip()

In [2]:
# need to test this
def seqs_from(seq, word_len):
    word_sets = []
    for start_pos in range(0,word_len):
        word_set = []
        for word_start_pos in range(start_pos, len(seq) - word_len + 1, word_len):
            word_set.append(seq[word_start_pos:(word_start_pos+word_len)])
        word_sets.append(word_set)
    return word_sets

In [3]:
# Step 2: set up dictionaries and transform data
def make_dictionaries(seq_generator, input_filename, word_len):
    word_count = dict()
    dictionary = dict()
    rev_dictionary = dict()
    index = 0
    for in_seq in seq_generator(input_filename):
        sub_seqs = seqs_from(in_seq, word_len)
        for seq in sub_seqs:
            for word in seq: # word is just an amino acid for now
                if word not in rev_dictionary.keys():
                    dictionary[index] = word
                    rev_dictionary[word] = index
                    word_count[word] = 1
                    index = index + 1
                else:
                    word_count[word] = word_count[word] + 1
        
    return dictionary, rev_dictionary, word_count

def gen_transformed_data(rev_dictionary, seq_generator, input_filename, word_len):
    for in_seq in seq_generator(input_filename):
        sub_seqs = seqs_from(in_seq, word_len)
        for seq in sub_seqs:
            transformed_data = []
            for word in seq: # word is just an amino acid for now
                transformed_data.append(rev_dictionary[word])
            yield transformed_data

In [4]:
# Step 3: make data generator and save everything to disk

def gen_seq_batch(seq, skip_window):
    min_index = 0
    max_index = len(seq) -1 
    batch = []
    labels = []
    for b_ind in range(max_index):
        for l_ind in range(b_ind-skip_window, b_ind+skip_window+1):
            if (l_ind < min_index) or (l_ind > max_index):
                pass
            elif b_ind == l_ind:
                pass
            else:
                batch.append(seq[b_ind])
                labels.append(seq[l_ind])
    return batch, labels

In [ ]:
def run_pre_process(input_filename='../data/protein/test.fasta',
                    dictionaries_filename='../data/protein/dictionaries',
                    batches_filename='../data/protein/batches',
                    skip_window=2,
                    down_sample_rate=100,
                    word_len=1):

    # save dictionaries
    dictionary, rev_dictionary, word_count = make_dictionaries(gen_seqs, input_filename, word_len)
    
    count_s = json.dumps(word_count)
    dict_s = json.dumps(dictionary)
    rev_dict_s = json.dumps(rev_dictionary)

    with open(dictionaries_filename + '_' + str(word_len) + '.json', 'w') as f_out:
        f_out.write(dict_s + '\n' + rev_dict_s + '\n' + count_s + '\n')

    # save data batches
    transformed_seqs = gen_transformed_data(rev_dictionary, gen_seqs, input_filename, word_len)
    with open(batches_filename + '_' + str(word_len) + '.csv', 'w') as f_out:
        for transformed_seq in transformed_seqs:
            ind = 0
            batch, labels = gen_seq_batch(transformed_seq, skip_window)
            for i in range(len(batch)):
                if (ind % down_sample_rate) == 0:
                    f_out.write(str(batch[i]) + ',' + str(labels[i]) + '\n')
                ind = ind + 1

In [ ]:
# 1-mers
p1 = Process(target=run_pre_process,
             kwargs=dict(input_filename='../data/protein/bacteria_nonredundant_protein.fasta',
                   skip_window=3,
                   down_sample_rate=100,
                   word_len=1))
p1.start()

# 2-mers
p2 = Process(target=run_pre_process,
             kwargs=dict(input_filename='../data/protein/bacteria_nonredundant_protein.fasta',
                   skip_window=1,
                   down_sample_rate=100,
                   word_len=2))
p2.start()

# 3-mers
p3 = Process(target=run_pre_process,
             kwargs=dict(input_filename='../data/protein/bacteria_nonredundant_protein.fasta',
                   skip_window=1,
                   down_sample_rate=100,
                   word_len=3))
p3.start()

p1.join()
p2.join()
p3.join()